In [48]:
import pandas as pd
import datetime
import time
import math

In [49]:
folder = '10_12_2018_07_55'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [50]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [51]:
res_eval['diff_p'] = (res_eval['pred_p'] - res_eval['real_p'])
res_eval['diff_p'] = res_eval['diff_p']*res_eval['diff_p']

In [52]:
res_eval.groupby('type')[['diff_p']].mean()

,diff_p
type,
CNN,0.183460
CRNN,0.018948
DNN,0.045226
FeatureBased_Bayesian Ridge Regression,0.052738
FeatureBased_Lasso Regression,0.052738
FeatureBased_Linear Regression,0.052738
FeatureBased_Passive Aggressive Regression,1.375208
FeatureBased_SGD Regression,12.138696
FeatureBased_SVM,0.052738


In [53]:
res_eval.groupby('type')[['real_p','pred_p']].corr().iloc[0::2,-1]

type                                              
CNN                                         real_p   -0.008745
CRNN                                        real_p    0.816894
DNN                                         real_p    0.379696
FeatureBased_Bayesian Ridge Regression      real_p    0.040260
FeatureBased_Lasso Regression               real_p    0.040260
FeatureBased_Linear Regression              real_p    0.040260
FeatureBased_Passive Aggressive Regression  real_p   -0.544085
FeatureBased_SGD Regression                 real_p   -0.546522
FeatureBased_SVM                            real_p    0.040260
FeatureBased_Theil-Sen Regression           real_p   -0.544875
GRU                                         real_p    0.398120
Name: pred_p, dtype: float64

In [54]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [55]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,4972,0,0.510157,0.572099,0.49122,0.118769,0.105026,0.041368,-0.391389,-0.467073,-0.449852
CNN,4974,0,0.510157,0.572099,0.49122,0.548083,0.543871,0.528613,0.037925,-0.028228,0.037393
CRNN,4976,0,0.510157,0.572099,0.49122,0.546002,0.541733,0.519127,0.035845,-0.030366,0.027907
DNN,4975,0,0.510157,0.572099,0.49122,0.545647,0.546403,0.537606,0.035489,-0.025696,0.046386
GRU,4973,0,0.510157,0.572099,0.49122,0.499866,0.503390,0.495137,-0.010291,-0.068709,0.003918
IR,4970,0,0.510157,0.572099,0.49122,0.494765,0.500000,0.497363,-0.015393,-0.072099,0.006144
SVDpp,4971,0,0.510157,0.572099,0.49122,0.496740,0.502001,0.489942,-0.013417,-0.070098,-0.001278


In [59]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [87]:
res_adapt['new_f_max'] = res_adapt.groupby(['pair', 'type'])['new_f'].transform(max)
clean_a = res_adapt[res_adapt['new_f_max'] == res_adapt['new_f']]
clean_a.groupby('type').mean()
# res_adapt['diff_f_max'] = res_adapt.groupby(['pair'])['diff_f'].transform(max)
# clean_a = res_adapt[res_adapt['diff_f_max'] == res_adapt['diff_f']]
# clean_a.groupby('type').mean()
# clean_a.groupby('type').mean()
# grouped = clean_a[clean_a['type'] == 'CNN'].groupby(['type', 'pair']).mean().reset_index()
# grouped.sort_values('diff_f', ascending=False)

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f,new_f_max,diff_f_max
type,,,,,,,,,,,,,
BPR,5097.556962,0.0,0.503642,0.604475,0.380223,0.498642,0.596188,0.374235,-0.005000,-0.008287,-0.005988,0.374235,0.505186
CNN,2213.125000,0.0,0.646726,0.739951,0.649259,0.736630,0.678272,0.687121,0.089905,-0.061679,0.037862,0.687121,0.504882
CRNN,2237.714286,0.0,0.632920,0.757791,0.639188,0.738150,0.687981,0.687572,0.105230,-0.069810,0.048384,0.687572,0.505061
DNN,3638.791667,0.0,0.582700,0.670790,0.575170,0.655283,0.630301,0.630868,0.072582,-0.040489,0.055698,0.630868,0.499269
GRU,4784.938298,0.0,0.512715,0.572360,0.491287,0.505761,0.506305,0.503156,-0.006954,-0.066055,0.011869,0.503156,0.494017
IR,4970.000000,0.0,0.510157,0.572099,0.491220,0.494765,0.500000,0.497363,-0.015393,-0.072099,0.006144,0.497363,0.505174
SVDpp,5785.703448,0.0,0.493292,0.542980,0.469315,0.521686,0.525716,0.518788,0.028394,-0.017264,0.049473,0.518788,0.498729


In [114]:
CRNN_adapt = res_adapt[res_adapt['type'] =='CRNN']

In [115]:
CRNN_adapt['new_f_max'] = CRNN_adapt.groupby(['pair'])['new_f'].transform(max)

C:\Users\shrag\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [116]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [117]:
CRNN_adapt_max['new_f'].mean()

0.6875723018379678

In [37]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [38]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder MWBG,GRU_GRU,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.643926,0.613711,1
1,1,1 AMC Token Path Ontobuilder MWBG,GRU_GRU,1,0.496382,0.500000,0.498184,0.496382,0.500000,0.498184,0.613711,0.643931,0.613711,1
2,2,1 AMC Token Path Ontobuilder MWBG,CNN_CNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.224611,0.613711,0
3,3,1 AMC Token Path Ontobuilder MWBG,DNN_DNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.641835,0.613711,0
4,4,1 AMC Token Path Ontobuilder MWBG,CRNN_CRNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.685498,0.613711,0
5,5,1 AMC Token Path Ontobuilder MWBG,GRU_CNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.224611,0.613711,0
6,6,1 AMC Token Path Ontobuilder MWBG,GRU_DNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.641835,0.613711,0
7,7,1 AMC Token Path Ontobuilder MWBG,GRU_CRNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.685498,0.613711,0
8,8,1 AMC Token Path Ontobuilder MWBG,CNN_GRU,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.643926,0.613711,1
9,9,1 AMC Token Path Ontobuilder MWBG,CNN_GRU,1,0.496382,0.500000,0.498184,0.496382,0.500000,0.498184,0.613711,0.643931,0.613711,1


In [39]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.510157,0.572099,0.491220
BPR_FEATLasso Regression,0.510157,0.572099,0.491220
BPR_FEATLinear Regression,0.510157,0.572099,0.491220
BPR_FEATPassive Aggressive Regression,0.510157,0.572099,0.491220
BPR_FEATSGD Regression,0.510157,0.572099,0.491220
BPR_FEATSVM,0.510157,0.572099,0.491220
BPR_FEATTheil-Sen Regression,0.510157,0.572099,0.491220
BPR_GRU,0.510157,0.572099,0.491220
CNN_CNN,0.548107,0.543853,0.528630


In [122]:
res_adapt_eval['pair'], res_adapt_eval['alg'] = res_adapt_eval['instance'].str.split(' ', 1).str
CRNN_adapt_eval = res_adapt_eval[res_adapt_eval['type'] =='CRNN']

In [123]:
CRNN_adapt_eval['new_f_max'] = CRNN_adapt_eval.groupby(['pair'])['new_f'].transform(max)

In [124]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [126]:
CRNN_adapt_max

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f,pair,alg,new_f_max,diff_f_max
27,27,10 WordNet Jiang Conrath Ontobuilder Threshold,CRNN,0,0.788051,0.747412,0.766128,0.768599,0.711698,0.736641,-0.019452,-0.035714,-0.029488,10,WordNet Jiang Conrath Ontobuilder Threshold,0.736641,0.482318
34,34,100 Ontobuilder Term Match Ontobuilder Max Delta,CRNN,0,0.672730,0.819886,0.722127,0.992771,0.666667,0.746359,0.320041,-0.153219,0.024232,100,Ontobuilder Term Match Ontobuilder Max Delta,0.746359,0.478918
41,41,100 Ontobuilder Term Match Ontobuilder Stable ...,CRNN,0,0.673621,0.766775,0.709396,0.992771,0.666667,0.746359,0.319150,-0.100109,0.036963,100,Ontobuilder Term Match Ontobuilder Stable Marr...,0.746359,0.478918
76,76,103 Ontobuilder Term Match Ontobuilder Dominants,CRNN,0,1.000000,1.000000,1.000000,0.999529,0.982143,0.990674,-0.000471,-0.017857,-0.009326,103,Ontobuilder Term Match Ontobuilder Dominants,0.990674,0.468421
104,104,104 Ontobuilder Term Match Ontobuilder Stable ...,CRNN,0,0.581442,0.693750,0.613567,0.898131,0.699688,0.765575,0.316689,0.005937,0.152008,104,Ontobuilder Term Match Ontobuilder Stable Marr...,0.765575,0.492270
167,167,105 WordNet Jiang Conrath Ontobuilder Max Delta,CRNN,0,0.642342,0.946523,0.710310,0.739493,0.955032,0.812265,0.097152,0.008509,0.101955,105,WordNet Jiang Conrath Ontobuilder Max Delta,0.812265,0.601569
195,195,107 Ontobuilder Term Match,CRNN,0,0.519573,0.758497,0.378473,0.911348,0.726378,0.791001,0.391775,-0.032119,0.412528,107,Ontobuilder Term Match,0.791001,0.476196
251,251,109 WordNet Jiang Conrath Ontobuilder MWBG,CRNN,0,0.597252,0.840891,0.649613,0.720943,0.653135,0.680823,0.123692,-0.187756,0.031210,109,WordNet Jiang Conrath Ontobuilder MWBG,0.680823,0.566905
279,279,11 WordNet Jiang Conrath Ontobuilder Dominants,CRNN,0,0.683388,0.683388,0.683388,0.795908,0.685855,0.727898,0.112520,0.002467,0.044510,11,WordNet Jiang Conrath Ontobuilder Dominants,0.727898,0.483912
314,314,111 Ontobuilder Term Match Ontobuilder Dominants,CRNN,0,0.660034,0.875952,0.722285,0.662957,0.650546,0.656495,0.002923,-0.225406,-0.065790,111,Ontobuilder Term Match Ontobuilder Dominants,0.656495,0.486832


In [34]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [35]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist